In [1]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.1 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# ✅ Safe Import for LSTM
try:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
except ImportError:
    from keras.models import Sequential
    from keras.layers import LSTM, Dense

st.title("📊 Sales/Demand Forecasting Dashboard")

uploaded_file = st.file_uploader("Upload your sales data CSV", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    df = df[['date','sales']]
    df.set_index('date', inplace=True)

    st.subheader("📈 Historical Sales Data")
    st.line_chart(df['sales'])

    FUTURE_DAYS = st.slider("Select Forecast Horizon (days):", 30, 180, 90)

    # ARIMA
    arima_model = ARIMA(df['sales'], order=(5,1,0))
    arima_fit = arima_model.fit()
    forecast_arima = arima_fit.forecast(steps=FUTURE_DAYS)

    # Prophet
    prophet_df = df.reset_index().rename(columns={'date':'ds','sales':'y'})
    prophet_model = Prophet()
    prophet_model.fit(prophet_df)
    future = prophet_model.make_future_dataframe(periods=FUTURE_DAYS)
    forecast_prophet = prophet_model.predict(future)

    # LSTM
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(df[['sales']])

    def create_sequences(data, seq_length):
        X, y = [], []
        for i in range(len(data)-seq_length):
            X.append(data[i:i+seq_length])
            y.append(data[i+seq_length])
        return np.array(X), np.array(y)

    SEQ_LEN = 30
    X, y = create_sequences(scaled_data, SEQ_LEN)
    X_train, y_train = X[:-FUTURE_DAYS], y[:-FUTURE_DAYS]

    model_lstm = Sequential()
    model_lstm.add(LSTM(64, activation='relu', input_shape=(SEQ_LEN,1)))
    model_lstm.add(Dense(1))
    model_lstm.compile(optimizer='adam', loss='mse')
    model_lstm.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

    last_seq = scaled_data[-SEQ_LEN:]
    future_preds = []
    current_seq = last_seq.reshape(1, SEQ_LEN, 1)

    for _ in range(FUTURE_DAYS):
        next_pred = model_lstm.predict(current_seq, verbose=0)[0][0]
        future_preds.append(next_pred)
        current_seq = np.append(current_seq[:,1:,:], [[[next_pred]]], axis=1)

    future_lstm = scaler.inverse_transform(np.array(future_preds).reshape(-1,1))

    st.subheader(f"🔮 Forecasts for Next {FUTURE_DAYS} Days")
    fig, ax = plt.subplots(figsize=(12,6))
    ax.plot(df.index, df['sales'], label="Historical Sales")
    ax.plot(pd.date_range(df.index[-1], periods=FUTURE_DAYS+1, freq="D")[1:], forecast_arima, label="ARIMA")
    ax.plot(forecast_prophet['ds'][-FUTURE_DAYS:], forecast_prophet['yhat'][-FUTURE_DAYS:], label="Prophet")
    ax.plot(pd.date_range(df.index[-1], periods=FUTURE_DAYS+1, freq="D")[1:], future_lstm, label="LSTM")
    ax.set_title("Future Demand Forecasting")
    ax.legend()
    st.pyplot(fig)


Writing app.py


In [3]:
from google.colab import files
files.download('app.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>